# Creating Types

## Outline
* Type synonyms
	* What
	* Why
	* Type Parameters 
* Defining new types
	* data
	* Value Parameters
	* Type Parameters
	* Record syntax

<div class="alert alert-block alert-warning">
<b>Warning:</b>
<p>We'll be adding <code>deriving (Show)</code> at the end of our data declarations.</p>
<p>By adding this, Haskell will automatically make that type part of the <code>Show</code> type class. Allowing us to print them on the terminal.</p>
<p>We'll explain how this works in detail on the next lesson.</p>
</div>

## Type Synonyms

Early on, when learning about `Strings` in Haskell, you found out that `String` is syntactic sugar for `[Char]`. They're the same but easier to read.

That's because `String` is a type synonym for `[Char]`.

This means that `String` and `[Char]` are *equivalent* and you can use them *interchangeably*.

### How to define Type Synonyms

```haskell
type String = [Char]
```

* The `type` keyword defines a type synonym.
* You can name the type synonym however you want, as long as it starts with a capital letter.

**When you define a type synonym, you're not creating a new type! You're telling Haskell that an existing type can be referred with a different name (a synonym)!**

### Why use Type Synonyms

Why would you add more complexity without adding more functionality?

Because type synonyms allow us to convey more information! Let's see an example.

Imagine you started working with a library that allows you to create monetary transactions.

You want to create a new transaction, so you take a look at the type signature of the function that you need to use:

```haskell
generateTx :: String -> String -> Int -> String 
```

Not an extremely useful signature. You could infer that the `Int` is the value to transfer, but what are those `Strings`? And what does that `String` that it returns contain?

Now, compare this type signature:

```haskell
generateTx :: String -> String -> Int -> String 
```

Whith this one:

```haskell
generateTx :: Address -> Address -> Value -> Id
```

Clearly, the second signature transmits the context way better!
The first two parameters are addresses, and the third one is the value of the transaction. Finally, we know that it returns the id of the transaction. 

All that just from the type signature. The difference? Just a few type synonyms.

And what did we do to improve the context so much? Let's see:

Let's recreate the function called `generateTx` that will take the addresses and value of a transaction and generates a id for it:

In [ ]:
generateTx :: String -> String -> Int -> String 
generateTx from to value = from ++ to ++ show value

Now, we just need to add some type synonyms:

In [ ]:
type Address = String
type Value = Int
type Id = String

generateTx :: Address -> Address -> Value -> Id
generateTx from to value = from ++ to ++ show value

Finally, if you want to check what does the `Address`, `Value`, or `Id` types are, you can open GHCi, load the file, and check its info:

In [ ]:
:i Address 
:i Value
:i Id

Another example:

In [ ]:
type Name = String
type Address = (String, Int)
type Person = (Name, Address)

bob = ("Bob Smith", ("Main St.", 555)) :: Person
:t bob
:t fst bob

Having type synonyms it's cool and all, but they are just different names for the same thing. What if we need to create a brand new type? `data` to the rescue!

## Defining new types

### `data` keyword

We can create new types using the `data` keyword, like this:

In [ ]:
data PaymentMethod = Cash | Card | Cryptocurrency

data Color = Red | Blue | Green

data Bool = True | False

* The part before the equal sign is our new type name.

* The part after the equal sign are **value constructors**. Value constructors specify the different **values** that the type can have.

* The `|` is read as "or". So we can read the firs type as:

		The type `PaymentMethod` can have a value of `Cash`, `Card`, or `Cryptocurrency`.

<div class="alert alert-block alert-warning">
<b>Warning:</b> The type name and the value constructors must start with an uppercase letter!
</div>

#### Using our new type

**And now, how can we use this new type?**

**By using its values!** For example, lets add a payment method to our person:

In [ ]:
type Name = String
type Address = (String, Int)

data PaymentMethod = Cash | Card | Cryptocurrency

type Person = (Name, Address, PaymentMethod)

bob = ("Bob Smith", ("Main St.", 555), Cash) :: Person

Now we can use the `PaymentMethod` type like any other type:

In [ ]:
howItPays :: Person -> String
howItPays (_, _, Cash) = "Pays in cash" -- Pattern matching Person and ignoring everything except the PaymentMethod
howItPays (_, _, Card) = "Pays with card"
howItPays (_, _, Cryptocurrency) = "Pays with cryptocurrency"

howItPays bob

But this generates a question. What can we do if we need more than a few values?

What if, for example, I want a type to represent a shape that could be any circle or any rectangle?

We can start by defining something like:

In [ ]:
data Shape = Circle | Rectangle

But the thing is, this isn't of much use.

I want to be able to do stuff with these values, like calculating perimeters and areas. And I can't do that without the actual properties of the shape!

**No problem at all! We can just add some parameters to the values!**

### Value Parameters

Let's add some parameters:
* To define a circle, we need its radius. So just one numeric value.
* To define a rectangle, we need the length of its two sides. So two numeric values.

Let's translate those requirements to code:

In [ ]:

data Shape = Circle Float | Rectangle Float Float

:t Circle
:t Rectangle

As you can see, if we check the type of the `Circle` constructor, we see that **it's actually a function!!**

A function that takes a value of type `Float` and returns a value of type `Shape`! So, to obtain our circle of type `Shape`, all we have to do is pass its radius:

In [ ]:
smallCircle = Circle 3

hugeCircle = Circle 100

Same for `Rectangle` values. 

`Rectangle` is a function that takes two values of type `Float` and returns a value of type `Shape`. So, to obtain a rectangle of type `Shape`, all we have to do is pass the lengths of its sides:

In [ ]:
rect1 = Rectangle 10 5

rect2 = Rectangle 256 128

That's it! We created our new type, and created some values. Now let's use them!

We can define a function that calculates the area of any value of type `Shape` like this:

In [ ]:
area :: Shape -> Float
area (Circle r) = pi * r^2        -- We pattern match on value constructors and bind its fields to names.
area (Rectangle l1 l2) = l1 * l2


area smallCircle
area rect2

Now we're talking! We just created an actually useful type!

But I'm not done with these shapes yet. I want more! I want to add points in space and colors!

We could do something like this monstrosity:

In [ ]:
data Shape = Circle (Float, Float) Float String | Rectangle (Float, Float) Float Float String

Where we add the points in space as tuples of `Float` and colors as `String`.

We could easily redefine the `area` function like this:

In [ ]:
area :: Shape -> Float
area (Circle _ r _) = pi * r^2
area (Rectangle _ l1 l2 _) = l1 * l2

But then, if we want to extract specific fields of the `Shape` type, we have to create a custom function for each and every one of them:

In [ ]:
color :: Shape -> String
color (Circle _ _ c) = c
color (Rectangle _ _ _ c) = c


point :: Shape -> (Float, Float)
point (Circle p _ _) = p
point (Rectangle p _ _ _) = p


--- Etc...

**That's just horrible!**
* Hard to read
* Hard to use
* Hard to extract the values from inside

And with what we know so far, I'm sure you're aware that we can add some type synonyms to improve at least the reading of the type:

In [ ]:
type Point = (Float,Float)
type Radius = Float
type Width = Float
type Height = Float
type Color = String

data Shape = Circle Point Radius Color | Rectangle Point Width Height Color

That's a lot of code to improve the understanding of the signature.

And on top of that, it doesn't solve the other two—arguably more pressing—problems!

But don't worry! Haskell has our backs! Enter the record syntax!

### Record Syntax

***Record syntax* is an alternative way of defining data types that comes with a few perks.**

We'll start with a simple example.

 Let's say we want to create a `Person` data type that contains these properties:

* Its name
* The company where it workd
* Years of experience
* Position

Without *record syntax*, we would create it like this:

In [ ]:
data Person String String Float String

But with *record syntax*, we can create it like this:

In [ ]:
data Person = Person { name :: String , company :: String , experience :: Float , position :: String } deriving (Show)

* Record syntax value constructors have their fields surrounded by curly brackets.
* Each field has a name that starts with a lowercase letter followed by its type.
* The fields are separated by commas.

To use this type, you just have to indicate the values of each field:

In [ ]:
richard = Person { name = "Richard", company = "Input Output", experience = 7.5, position = "Software Engineer"}

richard
:t richard

**And now, for the perks**

* The resulting data type is exactly the same.
* Easier to use and understand.
* The `Show` instance is more clear when we inspect it
* **It automatically generates functions to look up fields in the data type!**

In [ ]:
-- These funcitons are autogenerated when creating the Person data type:

:t name
name richard

:t experience
experience richard

As you can see, the auto-generated functions take a value of type `Person` and return the value of the field that has the same name as the function.

That's record syntax for you. 

To pick up the pace, let's redefine `Shape`, but now using record syntax:

In [ ]:
data Shape = Circle { position :: (Float, Float)
                    , radius :: Float
                    , color :: String
                    } | Rectangle {
                      position :: (Float, Float)
                    , width :: Float
                    , height :: Float
                    , color :: String
                    } deriving (Show)

We can use the data type the same as before:

In [ ]:
circ = Circle { position = (1,2), radius = 6, color = "Green"}
:t circ
circ

rect = Rectangle {position = (9,3), width = 7, height = 3, color = "Yellow"}
:t rect
rect

We can easily extract the values:

In [ ]:
position circ

color rect

And we can also pattern match!:

In [ ]:
area :: Shape -> Float
area (Circle {radius=r}) = pi * r^2
area (Rectangle {width=w,height=h}) = w * h

area circ
area rect

Ok. So far, we've defined several types. Both with values and value constructors (functions that take some parameters and produce a value).

But we can construct more than values. **We can construct types!**

### Paremeterizing Types

A **value** constructor **takes values** as parameters and **produces a value**.

                            |
                            v

A **type** constrctor **takes types** as parameters and **produces a type**.

We can use type constructors with both type synonyms and new types.

#### Parameterizing Type Synonyms

Going back to our last type synonym:

In [ ]:
type Name = String
type Address = (String, Int)
type Person = (Name, Address)

bob = ("Bob Smith", ("Main St.", 555)) :: Person

Imagine that after using it for a while, we find out that we also have to identify companies by their id number.

We could do something like:

In [ ]:
type Name = String
type Address = (String, Int)
type Person = (Name , Address)

type CompanyId = Int
type Company = (CompanyId, Address)

bob = ("Bob Smith", ("Main St.", 555)) :: Person
io = ("Input|Output", ("Cardano St.", 999)) :: Company

But if we do that, we repeat the same structure twice (`Person` and `Company`). A better aproach would be to define a parametric type synonym.

We add a parameter to the type synonym definition—same as with functions—and we can create different types for different situations!:

In [ ]:
type Name = String
type Address = (String, Int)
type CompanyId = Int

type Entity a = (a, Address)

bob = ("Bob Smith", ("Main St.", 555)) :: Entity Name 
io = (584264, ("Cardano St.", 999)) :: Entity CompanyId

Notice that:

* `Entity` by itself is a type constructor, not a type, so **no value can have a type of just `Entity`**.

* `Entity Name` and `Entity CompanyId` are completely different types!

For another example, we'll see how we can use type constructors when declaring new types.

### Parameterizing new types

To add type parameters while defining new types, you just do the same as with function and parameterized type synonyms:

Add the parameter to the left of the `=` sign, and (optionally) use it on the right:

In [ ]:
data Box a = Empty | Has a deriving (Show)

And to use the type:

In [ ]:
box1 = Has (1 :: Int)
:t box1

box2 = Has "Hello!"
:t box2

box3 = Empty
:t box3

We can also modify the values inside the boxes:

In [ ]:
add :: Num a => a -> Box a -> Box a
add _ Empty = Empty
add n (Has a) = Has (a + n)

box4 = add 3 box1
box4

We can also use type constructors with record syntax:

In [ ]:
data Shape a = Circle { position :: (Float, Float)
                      , radius :: Float
                      , color :: a
                      } | Rectangle {
                        position :: (Float, Float)
                      , width :: Float
                      , height :: Float
                      , color :: a
                      } deriving (Show)

In [ ]:
circleS = Circle { position = (1,2), radius = 6, color = "Green"}
:t circleS

circleRGB = Circle { position = (1,2), radius = 6, color = (0, 128, 0)}
:t circleRGB

**Examples of type constructors in base Haskell:**


The `Maybe` type constructor:

```haskell
data Maybe a = Nothing | Just a
```

The list type constructor:

```haskell
data [] a = [] | a : [a]
```

We'll learn more about `Maybe` in future lessons.

And that's it! Now you can create any type you want!